In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.2.0+cu92 (from versions: 1.4.0, 1.4.0+cpu, 1.4.0+cu100, 1.4.0+cu92, 1.5.0, 1.5.0+cpu, 1.5.0+cu101, 1.5.0+cu92, 1.5.1, 1.5.1+cpu, 1.5.1+cu101, 1.5.1+cu92, 1.6.0, 1.6.0+cpu, 1.6.0+cu101, 1.6.0+cu92, 1.7.0, 1.7.0+cpu, 1.7.0+cu101, 1.7.0+cu110, 1.7.0+cu92, 1.7.1, 1.7.1+cpu, 1.7.1+cu101, 1.7.1+cu110, 1.7.1+cu92, 1.7.1+rocm3.7, 1.7.1+rocm3.8, 1.8.0, 1.8.0+cpu, 1.8.0+cu101, 1.8.0+cu111, 1.8.0+rocm3.10, 1.8.0+rocm4.0.1, 1.8.1, 1.8.1+cpu, 1.8.1+cu101, 1.8.1+cu102, 1.8.1+cu111, 1.8.1+rocm3.10, 1.8.1+rocm4.0.1, 1.9.0, 1.9.0+cpu, 1.9.0+cu102, 1.9.0+cu111, 1.9.0+rocm4.0.1, 1.9.0+rocm4.1, 1.9.0+rocm4.2, 1.9.1, 1.9.1+cpu, 1.9.1+cu102, 1.9.1+cu111, 1.9.1+rocm4.0.1, 1.9.1+rocm4.1, 1.9.1+rocm4.2, 1.10.0, 1.10.0+cpu, 1.10.0+cu102, 1.10.0+cu111, 1.10.0+cu113, 1.10.0+roc

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
logging.basicConfig(level=logging.INFO)# OPTIONAL



tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
#model.to('cuda')  # if you have gpu


def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
predict_masked_sent("[MASK] time is mentioned in the sentence?", top_k=5)

[MASK]: 'what'  | weights: 0.8837916254997253
[MASK]: 'which'  | weights: 0.11069009453058243
[MASK]: 'whose'  | weights: 0.001109697506763041
[MASK]: 'the'  | weights: 0.0010072883451357484
[MASK]: 'when'  | weights: 0.0008617543498985469


In [ ]:
predict_masked_sent("[MASK] is the time ?", top_k=5)

[MASK]: 'when'  | weights: 0.6455521583557129
[MASK]: 'what'  | weights: 0.320957213640213
[MASK]: 'now'  | weights: 0.013798084110021591
[MASK]: 'where'  | weights: 0.011710303835570812
[MASK]: 'this'  | weights: 0.003477484453469515


In [ ]:
predict_masked_sent("[MASK] is the organization?", top_k=5)

[MASK]: 'where'  | weights: 0.6738113760948181
[MASK]: 'who'  | weights: 0.16383738815784454
[MASK]: 'what'  | weights: 0.14267708361148834
[MASK]: 'how'  | weights: 0.0065168049186468124
[MASK]: 'which'  | weights: 0.0033676994498819113


In [ ]:
predict_masked_sent("[MASK] organization is the  mentioned in the sentence?", top_k=5)

[MASK]: 'what'  | weights: 0.7173680067062378
[MASK]: 'which'  | weights: 0.15566961467266083
[MASK]: 'whose'  | weights: 0.11456398665904999
[MASK]: 'who'  | weights: 0.008127130568027496
[MASK]: 'the'  | weights: 0.0008716174634173512


In [ ]:
predict_masked_sent("[MASK] is the software mentioned in the sentence?", top_k=5)

[MASK]: 'what'  | weights: 0.5709157586097717
[MASK]: 'where'  | weights: 0.3097943663597107
[MASK]: 'who'  | weights: 0.08222436904907227
[MASK]: 'how'  | weights: 0.008232601918280125
[MASK]: 'why'  | weights: 0.008172501809895039


In [ ]:
predict_masked_sent("[MASK] software is mentioned in the sentence?", top_k=5)

[MASK]: 'what'  | weights: 0.8132886290550232
[MASK]: 'which'  | weights: 0.157545804977417
[MASK]: 'whose'  | weights: 0.015303007327020168
[MASK]: 'the'  | weights: 0.003413266269490123
[MASK]: 'who'  | weights: 0.002940273843705654
